<a href="https://colab.research.google.com/github/hjbcode/nodeproject/blob/main/Copy_of_Node_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Analyzing World Happiness

### Package and File Imports

In [ ]:
#Initial package imports
import pandas as pd
import numpy as np
import plotly.express as px
from google.colab import files
uploaded = files.upload()
import io

Saving 2015.csv to 2015 (3).csv
Saving 2016.csv to 2016 (3).csv
Saving 2017.csv to 2017 (3).csv
Saving 2018.csv to 2018 (3).csv
Saving 2019.csv to 2019 (3).csv


In [ ]:
#Data imports
happiness2015 = pd.read_csv(io.BytesIO(uploaded['2015.csv']))
happiness2016 = pd.read_csv(io.BytesIO(uploaded['2016.csv']))
happiness2017 = pd.read_csv(io.BytesIO(uploaded['2017.csv']))
happiness2018 = pd.read_csv(io.BytesIO(uploaded['2018.csv']))
happiness2019 = pd.read_csv(io.BytesIO(uploaded['2019.csv']))

### Data Cleaning

In [ ]:
# Cleaning 2015 data
columnnames = ["Country", "Rank", "Score", "GDPperCap", "SocialSup", "LifeExpect", "Freedom", "GovTrust", "Generosity", "Year"]
happiness2015 = happiness2015.drop(columns=["Region", "Standard Error", "Dystopia Residual"])
happiness2015['Year'] = 2015
happiness2015.columns = columnnames

In [ ]:
# Cleaning 2016 data
happiness2016 = happiness2016.drop(columns=["Region", "Lower Confidence Interval", "Upper Confidence Interval", "Dystopia Residual"])
happiness2016['Year'] = 2016
happiness2016.columns = columnnames

In [ ]:
# Cleaning 2017 data
happiness2017 = happiness2017.drop(columns=["Whisker.high", "Whisker.low", "Dystopia.Residual"])
move_generosity = happiness2017.pop("Generosity")
happiness2017.insert(8, "Generosity", move_generosity)
happiness2017['Year'] = 2017
happiness2017.columns = columnnames

In [ ]:
# Cleaning 2018 data
move_rank = happiness2018.pop("Overall rank")
move_generosity = happiness2018.pop("Generosity")
happiness2018.insert(1, "Overall rank", move_rank)
happiness2018.insert(8, "Generosity", move_generosity)
happiness2018['Year'] = 2018
happiness2018.columns = columnnames

In [ ]:
# Cleaning 2019 data
move_rank = happiness2019.pop("Overall rank")
move_generosity = happiness2019.pop("Generosity")
happiness2019.insert(1, "Overall rank", move_rank)
happiness2019.insert(8, "Generosity", move_generosity)
happiness2019['Year'] = 2019
happiness2019.columns = columnnames

In [ ]:
# Append all dataframes together
happiness = happiness2015.append(happiness2016).append(happiness2017).append(happiness2018).append(happiness2019)

In [ ]:
#Select countries that only appear in all 5 years
country = np.unique(happiness["Country"], return_counts = True)[0]
occurances = np.unique(happiness["Country"], return_counts = True)[1] == 5
dropcountries = country[np.where(occurances)]
happiness = happiness[happiness["Country"].isin(dropcountries)]

In [ ]:
#Install and import package for continents
!pip install pycountry_convert
import pycountry_convert as pc

In [ ]:
#Set dataframe index to Country so it's easier to drop countries
happiness = happiness.set_index("Country")

In [ ]:
#Drop countries that do not appear in alpha-2 country code list
happiness = happiness.drop("Kosovo").drop("Palestinian Territories").drop("Congo (Kinshasa)").drop("Congo (Brazzaville)")

In [ ]:
#Check for NAN values
happiness[happiness.isna().any(axis=1)]

,Rank,Score,GDPperCap,SocialSup,LifeExpect,Freedom,GovTrust,Generosity,Year
Country,,,,,,,,,
United Arab Emirates,20,6.774,2.096,0.776,0.67,0.284,NaN,0.186,2018


In [ ]:
#Drop UAE since it has NAN for GovTrust in 2018
happiness = happiness.drop("United Arab Emirates")

In [ ]:
#Assign continents
happiness["Country Code"] = happiness.index.map(lambda x: pc.country_name_to_country_alpha2(x, cn_name_format="default"))
happiness["Continent Code"] = happiness["Country Code"].map(lambda x: pc.country_alpha2_to_continent_code(x))
happiness["Continent"] = happiness["Continent Code"].map(lambda x: pc.convert_continent_code_to_continent_name(x))
happiness = happiness.drop(columns=["Country Code", "Continent Code"])

In [ ]:
#Cleaned Table
happiness

,Rank,Score,GDPperCap,SocialSup,LifeExpect,Freedom,GovTrust,Generosity,Year,Continent
Country,,,,,,,,,,
Switzerland,1,7.587,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2015,Europe
Iceland,2,7.561,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2015,Europe
Denmark,3,7.527,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2015,Europe
Norway,4,7.522,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2015,Europe
Canada,5,7.427,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2015,North America
...,...,...,...,...,...,...,...,...,...,...
Malawi,150,3.410,0.19100,0.56000,0.49500,0.44300,0.08900,0.21800,2019,Africa
Yemen,151,3.380,0.28700,1.16300,0.46300,0.14300,0.07700,0.10800,2019,Asia
Rwanda,152,3.334,0.35900,0.71100,0.61400,0.55500,0.41100,0.21700,2019,Africa


#### Data Standardization

In [ ]:
#Function for standardizing independent variables
def standard_units(arr):
    return (arr - np.mean(arr)) / np.std(arr)

In [ ]:
#Only standardize independent variables
happiness["StdScore"] = happiness["Score"]
happiness = happiness.apply(lambda x: standard_units(x) if x.name == "GDPperCap" or x.name == "SocialSup" or x.name == "Freedom"
                                or x.name == "LifeExpect" or x.name == "GovTrust" or x.name == "Generosity" or x.name == "StdScore" else x)

In [ ]:
#Final Standardized Table
happiness

,Rank,Score,GDPperCap,SocialSup,LifeExpect,Freedom,GovTrust,Generosity,Year,Continent,StdScore
Country,,,,,,,,,,,
Switzerland,1,7.587,1.175965,0.772814,1.313907,1.673203,2.757171,0.620291,2015,Europe,1.895770
Iceland,2,7.561,0.934549,0.934317,1.341304,1.426247,0.158212,1.724842,2015,Europe,1.872839
Denmark,3,7.527,0.993909,0.806726,1.028447,1.564556,3.352822,0.973459,2015,Europe,1.842853
Norway,4,7.522,1.336132,0.715957,1.073623,1.701120,2.245933,1.017793,2015,Europe,1.838444
Canada,5,7.427,0.995986,0.690408,1.160898,1.454433,1.914818,1.897507,2015,North America,1.754660
...,...,...,...,...,...,...,...,...,...,...,...
Malawi,150,3.410,-1.913851,-1.645782,-0.594135,0.179590,-0.331550,-0.003393,2019,Africa,-1.788082
Yemen,151,3.380,-1.667796,0.201456,-0.730903,-1.833638,-0.443602,-0.874240,2019,Asia,-1.814541
Rwanda,152,3.334,-1.483254,-1.183207,-0.085529,0.931195,2.675186,-0.011310,2019,Africa,-1.855110


### Data Vizualization

In [ ]:
#Happiness Score vs GDP
px.scatter(happiness, "GDPperCap", "StdScore", color = "Continent", hover_name = happiness.index, animation_frame="Year", animation_group=happiness.index, title="GDP per Capita on Happiness Score")

In [ ]:
#Happiness Score vs Social Support
px.scatter(happiness, "SocialSup", "StdScore", color = "Continent", hover_name = happiness.index, animation_frame="Year", animation_group=happiness.index, title="Social Support on Happiness Score")

In [ ]:
#Happiness Score vs Life Expectancy
px.scatter(happiness, "LifeExpect", "StdScore", color = "Continent", hover_name = happiness.index, animation_frame="Year", animation_group=happiness.index, title="Life Expectancy on Happiness Score")

In [ ]:
#Happiness Score vs Freedom
px.scatter(happiness, "Freedom", "StdScore", color = "Continent", hover_name = happiness.index, animation_frame="Year", animation_group=happiness.index, title="Freedom on Happiness Score")

In [ ]:
#Happiness Score vs Gov Trust
px.scatter(happiness, "GovTrust", "StdScore", color = "Continent", hover_name = happiness.index, animation_frame="Year", animation_group=happiness.index, title="Government Trust on Happiness Score")

In [ ]:
#Happiness Score vs Generosity
px.scatter(happiness, "Generosity", "StdScore", color = "Continent", hover_name = happiness.index, animation_frame="Year", animation_group=happiness.index, title="Generosity on Happiness Score")

### Machine Learning

In [ ]:
#ML imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
#Keep columns relavent for ML
ML_happiness = happiness.drop(columns=["Rank", "Year", "Continent", "StdScore"])

#Divide into X & y
X = ML_happiness.drop(columns=["Score"])
y = ML_happiness["Score"]
#Split data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
#Create and train model
mult_reg = LinearRegression()
mult_reg.fit(X_train,y_train)

LinearRegression()

In [ ]:
#Generate Predictions
predicted = mult_reg.predict(X_test)
actual = np.array(y_test)

#MSE & RMSE
mse = mean_squared_error(predicted,actual)
rmse = mean_squared_error(predicted,actual,squared=False)
print("MSE is ", round(mse,5), " and RMSE is ", round(rmse,5))

MSE is  0.31784  and RMSE is  0.56378


In [ ]:
#Beta Coefficients
coef = np.around(mult_reg.coef_,3)
intercept = round(mult_reg.intercept_,3)
factor_impact = (coef - coef[5]) / coef[5]

In [ ]:
print(factor_impact)

[4.50617284 1.38271605 2.97530864 1.44444444 0.2962963  0.        ]


In [ ]:
#Multiple Linear Regression Equation
print("Score =", coef[0],"*GDPperCap +", coef[1],"*SocialSup +",coef[2],"*LifeExpect +",coef[3],"*Freedom +",
      coef[4],"*GovTrust +",coef[5],"*Generosity +", intercept)

Score = 0.446 *GDPperCap + 0.193 *SocialSup + 0.322 *LifeExpect + 0.198 *Freedom + 0.105 *GovTrust + 0.081 *Generosity + 5.445


### Other Code

In [ ]:
countries = ""
for i in range(136):
  countries += happiness.index.unique()[i] + " | "
print(countries)

Switzerland | Iceland | Denmark | Norway | Canada | Finland | Netherlands | Sweden | New Zealand | Australia | Israel | Costa Rica | Austria | Mexico | United States | Brazil | Luxembourg | Ireland | Belgium | United Kingdom | Venezuela | Singapore | Panama | Germany | Chile | Qatar | France | Argentina | Czech Republic | Uruguay | Colombia | Thailand | Saudi Arabia | Spain | Malta | Kuwait | El Salvador | Guatemala | Uzbekistan | Slovakia | Japan | South Korea | Ecuador | Bahrain | Italy | Bolivia | Moldova | Paraguay | Kazakhstan | Slovenia | Lithuania | Nicaragua | Peru | Belarus | Poland | Malaysia | Croatia | Libya | Russia | Jamaica | Cyprus | Algeria | Turkmenistan | Mauritius | Estonia | Indonesia | Vietnam | Turkey | Kyrgyzstan | Nigeria | Bhutan | Azerbaijan | Pakistan | Jordan | Montenegro | China | Zambia | Romania | Serbia | Portugal | Latvia | Philippines | Morocco | Albania | Bosnia and Herzegovina | Dominican Republic | Mongolia | Greece | Lebanon | Hungary | Honduras |